In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [370]:
train_data.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

In [3]:
y=train_data.Survived
X=train_data.drop(['PassengerId', 'Survived'], axis=1)

In [4]:
num_cols = [col for col in X.columns if
           train_data[col].dtypes in ['int64', 'float64']]
cat_cols = [col for col in X.columns if 
           train_data[col].dtypes == 'object']

In [5]:
nX = X[num_cols]
cX = X[cat_cols]
from sklearn.preprocessing import Imputer
my_imputer=Imputer()
X_imputed = pd.DataFrame(my_imputer.fit_transform(nX))
X_imputed.columns = nX.columns

In [367]:
#from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OneHotEncoder
#encoded_x= None
#for i in range(0, cX.shape[1]):
  #  label_encoder = LabelEncoder()
   # feature = label_encoder.fit_transform(cX.values[:,i])
    #feature = feature.reshape(cX.shape[0],1)
    #onehot_encoder = OneHotEncoder(sparse=False)
    #feature = onehot_encoder.fit_transform(feature)
    #if encoded_x is None:
    #    encoded_x = feature
    #else:
    #    encoded_x = np.concatenate((encoded_x, feature), axis=1)
#print("cX shape: : ", encoded_x.shape)

TypeError: unorderable types: str() < float()

In [6]:
ohe_cX = pd.get_dummies(cX)

In [7]:
final_train = X_imputed.merge(ohe_cX, right_index=True, left_index=True)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
#score = cross_val_score(XGBClassifier(min_child_weight = 7.5, max_depth=5, n_estimator=700, eta=0.3), final_train1, y, scoring='accuracy')
#print(score.mean())

In [26]:
from sklearn.metrics import confusion_matrix
train_X, test_X, train_y, test_y = train_test_split(final_train, y)
model=XGBClassifier(n_estimator=900, learning_rate=0.2, min_child_weight=7)
model.fit(train_X, train_y, early_stopping_rounds=5, eval_set=[(test_X, test_y)], verbose=False)
prediction = model.predict(test_X)
#from sklearn.metrics import accuracy_score
#mae = accuracy_score(prediction, test_y)
#print(mae)
print(confusion_matrix(prediction, test_y))    

[[135  25]
 [ 11  52]]


F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [28]:
model.predict_proba(test_X)[0:10]

array([[0.77621543, 0.22378454],
       [0.595224  , 0.404776  ],
       [0.77621543, 0.22378454],
       [0.17660958, 0.8233904 ],
       [0.77621543, 0.22378454],
       [0.77621543, 0.22378454],
       [0.6569369 , 0.34306312],
       [0.17660958, 0.8233904 ],
       [0.77621543, 0.22378454],
       [0.77621543, 0.22378454]], dtype=float32)

In [388]:
test_data1=test_data.drop(['PassengerId'], axis=1)
ntest = test_data1[num_cols]
ctest=test_data1[cat_cols]

In [389]:
imputed_test = pd.DataFrame(my_imputer.fit_transform(ntest))
imputed_test.columns = ntest.columns
ohe_ctest = pd.get_dummies(ctest)
final_test = imputed_test.merge(ohe_ctest, right_index=True, left_index=True)

In [394]:
print(final_train1.shape)
print(final_test1.shape)

(891, 2432)
(418, 2432)


In [393]:
final_train1, final_test1 = final_train.align(final_test, join='outer', axis=1)

In [458]:
model= XGBClassifier(n_estimator=900, eta=0.05)
model.fit(final_train1, y)
prediction=model.predict(final_test1)


F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [299]:
test_data.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [470]:
submission=pd.DataFrame({'PassengerId':test_data.PassengerId, 'Survived':prediction})

In [471]:
submission.to_csv('Predict.csv', index=False)